<a href="https://colab.research.google.com/github/krishna7356/pandas/blob/gen-AI/Welcome_To_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
import json
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, pipeline

In [6]:
reviews = [
    # Review 1: Positive
    "I absolutely love the new QuantumX Pro camera! The picture quality is stellar and the battery life is amazing. Shipped super fast too. A++!",

    # Review 2: Negative with specific issue
    "The SonicWave earbuds have a serious design flaw. The left earbud stopped charging after just one week. I expected better for the price. Very disappointed.",

    # Review 3: Mixed with a question
    "The Titan smartwatch is decent. The screen is bright and the features are good, but the step counter seems inaccurate. It's off by at least 20%. Is there a way to calibrate it?",

    # Review 4: Negative with multiple issues
    "My order for the AeroDrone was a disaster. It arrived with a broken propeller and the battery was completely dead on arrival. Customer service has been unresponsive for 3 days.",

    # Review 5: Positive but mentions a minor issue
    "Overall, I'm happy with the PureGlow Air Purifier. It's quiet and effective. My only complaint is that the replacement filters are a bit expensive."
]

In [2]:



# Load GPT-2 tokenizer
gpt2_tokenizer = AutoTokenizer.from_pretrained("gpt2")

# Load BERT-base-uncased tokenizer
bert_tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")


print("GPT-2 Tokenized:", gpt2_tokenizer.tokenize(reviews[3]))
print("BERT Tokenized:", bert_tokenizer.tokenize(reviews[3]))


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

GPT-2 Tokenized: ['My', 'Ġorder', 'Ġfor', 'Ġthe', 'ĠAero', 'Dr', 'one', 'Ġwas', 'Ġa', 'Ġdisaster', '.', 'ĠIt', 'Ġarrived', 'Ġwith', 'Ġa', 'Ġbroken', 'Ġprope', 'ller', 'Ġand', 'Ġthe', 'Ġbattery', 'Ġwas', 'Ġcompletely', 'Ġdead', 'Ġon', 'Ġarrival', '.', 'ĠCustomer', 'Ġservice', 'Ġhas', 'Ġbeen', 'Ġun', 'responsive', 'Ġfor', 'Ġ3', 'Ġdays', '.']
BERT Tokenized: ['my', 'order', 'for', 'the', 'aero', '##dron', '##e', 'was', 'a', 'disaster', '.', 'it', 'arrived', 'with', 'a', 'broken', 'propeller', 'and', 'the', 'battery', 'was', 'completely', 'dead', 'on', 'arrival', '.', 'customer', 'service', 'has', 'been', 'un', '##res', '##pon', '##sive', 'for', '3', 'days', '.']


**TASK 1**

In [22]:
def build_few_shot_prompt(review_text):

  prompt = f"""
You are a Customer Feedback Analyst in an e-commerce company.Your role is to help the customer support team quickly understand customer feedback.
Carefully read each review and consider both positive and negative aspects before classifying the sentiment.
Classify the sentiment of the following product reviews as Positive, Negative, or Mixed.
Output only one word: Positive, Negative, or Mixed.

Examples:

Review: "I absolutely love the new QuantumX Pro camera! The picture quality is stellar and the battery life is amazing. Shipped super fast too. A++!"
Sentiment: Positive

Review: "The Titan smartwatch is decent. The screen is bright and the features are good, but the step counter seems inaccurate. It's off by at least 20%. Is there a way to calibrate it?"
Sentiment: Mixed

Now classify this review:
"{review_text}"
Sentiment:
"""

  return prompt

In [5]:

# Model
model_name = "google/flan-t5-large"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Pipeline for text2text generation
classifier = pipeline(
    task="text2text-generation",
    model=model,
    tokenizer=tokenizer,
    device=0,
)


tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Device set to use cpu


In [29]:
fewshort_results = []

for review in reviews:
    prompt = build_few_shot_prompt(review)

    # Generate deterministic output
    output = classifier(prompt, do_sample=False)[0]['generated_text']

    # Extract first word as final label
    label = output.strip().split()[0].capitalize()

    fewshort_results.append({"review": review, "sentiment": label})

# Print results
for r in fewshort_results:
    print(f"Review: {r['review']}\nPredicted Sentiment: {r['sentiment']}\n")


Review: I absolutely love the new QuantumX Pro camera! The picture quality is stellar and the battery life is amazing. Shipped super fast too. A++!
Predicted Sentiment: Positive

Review: The SonicWave earbuds have a serious design flaw. The left earbud stopped charging after just one week. I expected better for the price. Very disappointed.
Predicted Sentiment: Negative

Review: The Titan smartwatch is decent. The screen is bright and the features are good, but the step counter seems inaccurate. It's off by at least 20%. Is there a way to calibrate it?
Predicted Sentiment: Mixed

Review: My order for the AeroDrone was a disaster. It arrived with a broken propeller and the battery was completely dead on arrival. Customer service has been unresponsive for 3 days.
Predicted Sentiment: Negative

Review: Overall, I'm happy with the PureGlow Air Purifier. It's quiet and effective. My only complaint is that the replacement filters are a bit expensive.
Predicted Sentiment: Positive



**TASK 2**

In [49]:
def build_extraction_prompt(review_text):
    prompt = f"""
You are a Customer Feedback Analyst in an e-commerce company.
Your role is to help the customer support team quickly understand customer feedback.
Extract structured information from each customer review.
Return the output in exact JSON format with keys: "product_name", "issue_summary", "sentiment".
- "product_name": the product mentioned. If not mentioned, write "N/A".
- "issue_summary": include all negative points, complaints, or problems. If none, write "N/A".
- "sentiment": classify the overall review as Positive, Negative, or Mixed after analyzing pros and cons carefully.

Examples:

Review: "I absolutely love the new QuantumX Pro camera! The picture quality is stellar and the battery life is amazing. Shipped super fast too. A++!"
Output:
{{"product_name": "QuantumX Pro camera", "issue_summary": "N/A", "sentiment": "Positive"}}

Review: "The SonicWave earbuds have a serious design flaw. The left earbud stopped charging after just one week. I expected better for the price. Very disappointed."
Output:
{{"product_name": "SonicWave earbuds", "issue_summary": "left earbud stopped charging, design flaw", "sentiment": "Negative"}}

Review: "The Titan smartwatch is decent. The screen is bright and the features are good, but the step counter seems inaccurate. It's off by at least 20%. Is there a way to calibrate it?"
Output:
{{"product_name": "Titan smartwatch", "issue_summary": "step counter inaccurate", "sentiment": "Mixed"}}

Now extract structured data from this review:
"{review_text}"
Output:
"""
    return prompt


In [50]:
prompt = build_extraction_prompt(reviews[4])
output = classifier(prompt, do_sample=False)
print(output)

[{'generated_text': '"product_name": "PureGlow Air Purifier", "issue_summary": "It\'s quiet and effective. It\'s only complaint is that the replacement filters are a bit expensive"'}]


In [48]:
extraction_results = []

for review in reviews:
    prompt = build_extraction_prompt(review)

    # Generate output
    output = classifier(prompt, do_sample=False)

    extraction_results.append(output)

ValueError: Invalid format specifier ' "QuantumX Pro camera", "issue_summary": "N/A", "sentiment": "Positive"' for object of type 'str'

In [44]:
# Step 6: Print results
for i, r in enumerate(extraction_results, 1):
    print(r)

[{'generated_text': '"product_name": "QuantumX Pro camera", "issue_summary": "N/A", "sentiment": "Positive"'}]
[{'generated_text': '"product_name": "SonicWave earbuds", "issue_summary": "left earbud stopped charging, design flaw", "sentiment": "Negative"'}]
[{'generated_text': '"product_name": "Titan smartwatch", "issue_summary": "N/A", "sentiment": "Negative"'}]
[{'generated_text': '"product_name": "AeroDrone", "issue_summary": "...," "sentiment": "Negative"'}]
[{'generated_text': '"product_name": "PureGlow Air Purifier", "issue_summary": "N/A", "sentiment": "Negative"'}]
